In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from vacances_scolaires_france import SchoolHolidayDates
import datetime
pd.set_option('display.max_columns', 500)



# df = pd.read_csv('../films_db2.csv')
# df.head(10)

In [2]:
# from db.database_mysql import engine

# df = pd.read_sql_query('''SELECT jp.raw_title, 
#         YEAR(jp.date) AS year, 
#         MONTH(jp.date) AS month, 
#         DAY(jp.date) AS day, 
#     im.director, im.date,im.casting, im.distributor, jp.country, jp.duration, jp.genre, jp.first_day, jp.first_week, jp.first_weekend, jp.hebdo_rank, 
# jp.total_spectator, jp.copies, im.rating_press, im.budget, im.lang, im.award 
# FROM films_jp as jp
# LEFT JOIN films_imdb im ON im.id_jp = jp.id 
# where im.id_jp is not null and im.date = jp.date
# order by jp.first_week desc''', engine)

# df

In [3]:
from db.database_mysql import engine

df = pd.read_sql_query('''SELECT jp.raw_title, 
        YEAR(jp.date) AS year, 
        MONTH(jp.date) AS month, 
        DAY(jp.date) AS day, 
        im.director, im.date,im.casting, im.distributor, im.genre, jp.country, jp.duration, jp.first_day, jp.first_week, jp.first_weekend, jp.hebdo_rank, 
jp.total_spectator, jp.copies, im.rating_press, im.budget, im.lang, im.award 
FROM films_jp as jp
LEFT JOIN films_imdb im ON im.id_jp = jp.id 
where im.id_jp is not null and im.date = jp.date
order by jp.first_week desc''', engine)

df

,raw_title,year,month,day,director,date,casting,distributor,genre,country,duration,first_day,first_week,first_weekend,hebdo_rank,total_spectator,copies,rating_press,budget,lang,award
0,Bienvenue chez les Ch'tis,2008,2,27,"""dany boon""",2008-02-27,"[""kad merad"", ""dany boon"", ""zoe felix""]","[""pathe renn productions"", ""hirsch"", ""les prod...","[""comedie"", ""romantique""]",france,6360,558359,4378720,3586497,1,20489303,793,7.1,11000000,"[""francais""]",5
1,Les Bronzés 3: Amis pour la vie,2006,2,1,"""patrice leconte""",2006-02-01,"[""josiane balasko"", ""michel blanc"", ""marieanne...","[""les films christian fechner"", ""tf1 films pro...","[""comedie""]",france,5820,537882,3906694,3235559,1,10355930,950,-1.0,35000000,"[""anglais"", ""italien"", ""francais""]",1
2,Taxi 2,2000,3,29,"""gerard krawczyk""",2000-03-29,"[""samy naceri"", ""frederic diefenthal"", ""emma w...","[""arp selection"", ""canal"", ""leeloo productions""]","[""action"", ""comedie"", ""policier""]",france,5400,801922,3478850,2951255,1,10345901,831,6.5,70000000,"[""francais"", ""japonais"", ""allemand""]",1
3,La Revanche des Sith,2005,5,18,"""george lucas""",2005-05-18,"[""hayden christensen"", ""natalie portman"", ""ewa...","[""lucasfilm"", ""mestiere cinema"", ""pandora films""]","[""action"", ""aventure"", ""fantastique""]",etatsunis,8760,641799,3303005,2878764,1,7247809,938,7.6,113000000,"[""anglais""]",29
4,Le Roi Lion (2019),2019,7,17,"""jon favreau""",2019-07-17,"[""donald glover"", ""beyonce"", ""seth rogen""]","[""walt disney pictures"", ""fairview entertainme...","[""animation"", ""aventure"", ""drame""]",etatsunis,7080,630478,3252896,2559370,1,10017995,680,6.8,260000000,"[""anglais"", ""espagnol""]",21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2797,Rocks,2020,9,9,"""sarah gavron""",2020-09-09,"[""bukky bakray"", ""kosar ali"", ""dangelou osei k...","[""fable pictures"", ""british film institute bfi...","[""drame""]",grandebretagne,5580,1989,10895,8820,18,27284,114,7.4,-1,"[""anglais""]",16
2798,Chercher le garçon,2012,5,9,"""dorothee sebbagh""",2012-05-09,"[""sophie cattani"", ""moussa maaskri"", ""gerard d...","[""belavox films"", ""moby dick films"", ""producti...","[""drame""]",france,4200,886,10617,8123,19,26290,33,5.8,-1,"[""francais""]",0
2799,Sauve moi,2000,9,6,"""christian vincent""",2000-09-06,"[""roschdy zem"", ""rona hartner"", ""karole rocher""]","[""agat films cie"", ""crrav"", ""canal""]","[""drame""]",france,6000,-1,10555,-1,18,26541,42,5.8,-1,"[""francais""]",0
2800,Mari Iyagi,2003,5,28,"""seonggang lee""",2003-05-28,"[""lee byunghun"", ""kong hyeongjin"", ""bae jongok""]","[""ipictures"", ""muhan investment"", ""korean film...","[""animation"", ""drame"", ""fantastique""]",coree du sud,4800,1033,10051,7530,19,58256,29,6.5,-1,"[""anglais"", ""coreen""]",1


In [4]:
df.isnull().sum()

raw_title            0
year                 0
month                0
day                  0
director             0
date                 0
casting              0
distributor          0
genre              154
country              0
duration             0
first_day            0
first_week           0
first_weekend        0
hebdo_rank           0
total_spectator      0
copies               0
rating_press         0
budget               0
lang                 0
award                0
dtype: int64

In [5]:
df = df.dropna()

In [6]:
df.nunique()

raw_title          2646
year                 32
month                12
day                  31
director           1513
date               1346
casting            2616
distributor        2451
genre               277
country              46
duration            118
first_day          2192
first_week         2637
first_weekend      2072
hebdo_rank           20
total_spectator    2642
copies              764
rating_press         58
budget              392
lang                566
award               108
dtype: int64

In [7]:
# def calculate_director_scores(df):
#     # 1. Fréquence de réalisation
#     frequency = df['director'].value_counts()

#     # 2. Moyenne des spectateurs totaux
#     avg_total_spectators = df.groupby('director')['total_spectator'].mean()

#     # 3. Moyenne de la première semaine
#     avg_first_week = df.groupby('director')['first_week'].mean()

#     # 4. Moyenne inversée du classement hebdomadaire
#     avg_inv_hebdo_rank = 1 / df.groupby('director')['hebdo_rank'].mean()

#     # 5. Moyenne des évaluations de la presse
#     avg_rating_press = df.groupby('director')['rating_press'].mean()

#     # 5. Moyenne des récompenses
#     avg_award = df.groupby('director')['award'].mean()

#     # 6. Rendement (total_spectator / budget)
#     performance = (df.groupby('director')['total_spectator'].sum() / df.groupby('director')['budget'].sum())

#     # Compilation des scores dans un DataFrame
#     director_scores = pd.DataFrame({
#         'frequency': frequency,
#         'avg_total_spectators': avg_total_spectators,
#         'avg_first_week': avg_first_week,
#         'avg_inv_hebdo_rank': avg_inv_hebdo_rank,
#         'avg_rating_press': avg_rating_press,
#         'avg_award': avg_award,
#         'performance': performance
#     })

#     # Normalisation des scores
#     max_values = director_scores.max()
#     normalized_scores = director_scores / max_values

#     # Combinaison des scores normalisés
#     normalized_scores['director_combined_score'] = normalized_scores.mean(axis=1)

#     return normalized_scores

# # Appliquer la fonction pour calculer les scores
# director_scores = calculate_director_scores(df)

# # Joindre les scores combinés avec le DataFrame original
# df = df.join(director_scores['director_combined_score'], on='director')
# df 


In [8]:


# def classify_entrees_year(entrees):
#     if entrees == 1986:
#         return 168.1
#     elif entrees == 1992:
#         return 116.0
#     elif entrees == 1993:
#         return 132.7
#     elif entrees == 1994:
#         return 124.4
#     elif entrees == 1995:
#         return 130.2
#     elif entrees == 1996:
#         return 136.7
#     elif entrees == 1997:
#         return 149.3
#     elif entrees == 1998:
#         return 170.6
#     elif entrees == 1999:
#         return 153.6
#     elif entrees == 2000:
#         return 165.8
#     elif entrees == 2001:
#         return 187.5
#     elif entrees == 2002:
#         return 184.4
#     elif entrees == 2003:
#         return 173.5
#     elif entrees == 2004:
#         return 195.8
#     elif entrees == 2005:
#         return 175.6
#     elif entrees == 2006:
#         return 188.8
#     elif entrees == 2007:
#         return 178.5
#     elif entrees == 2008:
#         return 190.3
#     elif entrees == 2009:
#         return 201.6
#     elif entrees == 2010:
#         return 207.1
#     elif entrees == 2011:
#         return 217.2
#     elif entrees == 2012:
#         return 203.6
#     elif entrees == 2013:
#         return 193.7
#     elif entrees == 2014:
#         return 209.1
#     elif entrees == 2015:
#         return 205.4
#     elif entrees == 2016:
#         return 213.2
#     elif entrees == 2017:
#         return 209.4
#     elif entrees == 2018:
#         return 201.2
#     elif entrees == 2019:
#         return 213.2
#     elif entrees == 2020:
#         return 65.3
#     elif entrees == 2021:
#         return 95.5
#     elif entrees == 2022:
#         return 	152.0
#     elif entrees == 2023 or 2024:
#         return 180.8



# df['entree_annee'] = df['year'].apply(classify_entrees_year)

# df

In [9]:
f_acteurs = pd.DataFrame(df['casting'].str.strip('[]').str.split(',').explode().unique(), columns=['actor'])
f_acteurs

,actor
0,"""kad merad"""
1,"""dany boon"""
2,"""zoe felix"""
3,"""josiane balasko"""
4,"""michel blanc"""
...,...
4237,"""gerard dubouche"""
4238,"""rona hartner"""
4239,"""lee byunghun"""
4240,"""kong hyeongjin"""


In [10]:
actor_counts = df['casting'].str.strip('[]').str.split(',').explode().value_counts()
# Convertir les acteurs en listes à partir des chaînes de caractères
df['actor_list'] = df['casting'].str.strip('[]').str.split(',')

# Exploder cette colonne pour avoir une ligne par acteur par film
expanded_df = df.explode('actor_list')

# Calculer la moyenne des évaluations de la presse pour chaque acteur
avg_rating_by_actor = expanded_df.groupby('actor_list')['rating_press'].mean()
performance_by_actor = expanded_df.groupby('actor_list').apply(
    lambda x: (x['total_spectator'].sum() / x['budget'].sum()) if x['budget'].sum() > 0 else 0)

# Création d'un DataFrame pour les scores
scores = pd.DataFrame({
    'frequency': actor_counts,
    'avg_rating_press': avg_rating_by_actor,
    'performance': performance_by_actor
})

# Normalisation des scores (chaque colonne sera divisée par sa valeur maximale)
normalized_scores = scores.apply(lambda x: x / x.max())

# Calcul du score combiné
normalized_scores['combined_score'] = normalized_scores.mean(axis=1)
f_acteurs = f_acteurs.join(normalized_scores['combined_score'], on='actor')
f_acteurs['actor'] = f_acteurs['actor'].str.replace('"', '')

f_acteurs


/tmp/ipykernel_567589/88953483.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['actor_list'] = df['casting'].str.strip('[]').str.split(',')
/tmp/ipykernel_567589/88953483.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  performance_by_actor = expanded_df.groupby('actor_list').apply(


,actor,combined_score
0,kad merad,0.272672
1,dany boon,0.351076
2,zoe felix,0.237638
3,josiane balasko,0.334050
4,michel blanc,0.232019
...,...,...
4237,gerard dubouche,0.224552
4238,rona hartner,0.224552
4239,lee byunghun,0.249642
4240,kong hyeongjin,0.249642


In [11]:
# Créer un dictionnaire pour accéder plus facilement aux scores
actor_scores_dict = f_acteurs.set_index('actor')['combined_score'].to_dict()
actor_scores_dict

{'kad merad': 0.2726718782382913,
 ' dany boon': 0.3510758426553681,
 ' zoe felix': 0.23763767202008781,
 'josiane balasko': 0.33405034868296885,
 ' michel blanc': 0.23201937764704006,
 ' marieanne chazel': 0.14199579510140592,
 'samy naceri': 0.23500633249108857,
 ' frederic diefenthal': 0.2720433682328191,
 ' emma wiklund': 0.24964181316505787,
 'hayden christensen': 0.2890682028201318,
 ' natalie portman': 0.32491049562947166,
 ' ewan mcgregor': 0.35988227519541466,
 'donald glover': 0.2603943267847753,
 ' beyonce': 0.2663082967148945,
 ' seth rogen': 0.27706098628527237,
 'jamie bell': 0.3116488332389203,
 ' andy serkis': 0.3234767498692422,
 ' daniel craig': 0.27831547501419085,
 'daniel radcliffe': 0.3859959787348724,
 ' emma watson': 0.3931644362383593,
 ' rupert grint': 0.38753207732315736,
 'gerard depardieu': 0.4081841044139851,
 ' clovis cornillac': 0.3543013355719124,
 ' benoit poelvoorde': 0.3147851810669838,
 'elijah wood': 0.23745527737660518,
 ' viggo mortensen': 0.2259

In [12]:
def sum_actor_scores(casting):
    # Assumer que casting est une liste d'acteurs déjà évaluée si nécessaire
    total_score = 0
    for actor in casting:
        # Enlever les guillemets et les espaces superflus
        actor = actor.strip('"')
        # Ajouter le score de l'acteur s'il est connu, sinon ajouter 0
        total_score += actor_scores_dict.get(actor, 0)
    return total_score

# Appliquer la fonction en évaluant la liste d'acteurs si nécessaire
df['total_actor_scores'] = df['casting'].apply(lambda x: sum_actor_scores(eval(x)))
df

/tmp/ipykernel_567589/1795014875.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_actor_scores'] = df['casting'].apply(lambda x: sum_actor_scores(eval(x)))


,raw_title,year,month,day,director,date,casting,distributor,genre,country,duration,first_day,first_week,first_weekend,hebdo_rank,total_spectator,copies,rating_press,budget,lang,award,actor_list,total_actor_scores
0,Bienvenue chez les Ch'tis,2008,2,27,"""dany boon""",2008-02-27,"[""kad merad"", ""dany boon"", ""zoe felix""]","[""pathe renn productions"", ""hirsch"", ""les prod...","[""comedie"", ""romantique""]",france,6360,558359,4378720,3586497,1,20489303,793,7.1,11000000,"[""francais""]",5,"[""kad merad"", ""dany boon"", ""zoe felix""]",0.559846
1,Les Bronzés 3: Amis pour la vie,2006,2,1,"""patrice leconte""",2006-02-01,"[""josiane balasko"", ""michel blanc"", ""marieanne...","[""les films christian fechner"", ""tf1 films pro...","[""comedie""]",france,5820,537882,3906694,3235559,1,10355930,950,-1.0,35000000,"[""anglais"", ""italien"", ""francais""]",1,"[""josiane balasko"", ""michel blanc"", ""mariean...",0.603943
2,Taxi 2,2000,3,29,"""gerard krawczyk""",2000-03-29,"[""samy naceri"", ""frederic diefenthal"", ""emma w...","[""arp selection"", ""canal"", ""leeloo productions""]","[""action"", ""comedie"", ""policier""]",france,5400,801922,3478850,2951255,1,10345901,831,6.5,70000000,"[""francais"", ""japonais"", ""allemand""]",1,"[""samy naceri"", ""frederic diefenthal"", ""emma...",0.434469
3,La Revanche des Sith,2005,5,18,"""george lucas""",2005-05-18,"[""hayden christensen"", ""natalie portman"", ""ewa...","[""lucasfilm"", ""mestiere cinema"", ""pandora films""]","[""action"", ""aventure"", ""fantastique""]",etatsunis,8760,641799,3303005,2878764,1,7247809,938,7.6,113000000,"[""anglais""]",29,"[""hayden christensen"", ""natalie portman"", ""e...",0.964068
4,Le Roi Lion (2019),2019,7,17,"""jon favreau""",2019-07-17,"[""donald glover"", ""beyonce"", ""seth rogen""]","[""walt disney pictures"", ""fairview entertainme...","[""animation"", ""aventure"", ""drame""]",etatsunis,7080,630478,3252896,2559370,1,10017995,680,6.8,260000000,"[""anglais"", ""espagnol""]",21,"[""donald glover"", ""beyonce"", ""seth rogen""]",0.524373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2797,Rocks,2020,9,9,"""sarah gavron""",2020-09-09,"[""bukky bakray"", ""kosar ali"", ""dangelou osei k...","[""fable pictures"", ""british film institute bfi...","[""drame""]",grandebretagne,5580,1989,10895,8820,18,27284,114,7.4,-1,"[""anglais""]",16,"[""bukky bakray"", ""kosar ali"", ""dangelou osei...",0.281900
2798,Chercher le garçon,2012,5,9,"""dorothee sebbagh""",2012-05-09,"[""sophie cattani"", ""moussa maaskri"", ""gerard d...","[""belavox films"", ""moby dick films"", ""producti...","[""drame""]",france,4200,886,10617,8123,19,26290,33,5.8,-1,"[""francais""]",0,"[""sophie cattani"", ""moussa maaskri"", ""gerard...",0.224552
2799,Sauve moi,2000,9,6,"""christian vincent""",2000-09-06,"[""roschdy zem"", ""rona hartner"", ""karole rocher""]","[""agat films cie"", ""crrav"", ""canal""]","[""drame""]",france,6000,-1,10555,-1,18,26541,42,5.8,-1,"[""francais""]",0,"[""roschdy zem"", ""rona hartner"", ""karole roch...",0.322402
2800,Mari Iyagi,2003,5,28,"""seonggang lee""",2003-05-28,"[""lee byunghun"", ""kong hyeongjin"", ""bae jongok""]","[""ipictures"", ""muhan investment"", ""korean film...","[""animation"", ""drame"", ""fantastique""]",coree du sud,4800,1033,10051,7530,19,58256,29,6.5,-1,"[""anglais"", ""coreen""]",1,"[""lee byunghun"", ""kong hyeongjin"", ""bae jong...",0.249642


In [13]:
# df['genre'] = df['genre'].apply(lambda x: x.split()[0] if x else None)
# df['genre'] = df['genre'].str.replace('"', '')
# df['genre'] = df['genre'].str.replace('[', '')
# df['genre'] = df['genre'].str.replace(']', '')
# df['genre'] = df['genre'].str.replace(',', '')


In [14]:
# def classifier_pays(pays):
#     if pays == 'etatsunis':
#         return 3
#     elif pays == 'france':
#         return 2
#     else :
#         return 0

# df['classification_country'] = df['country'].apply(lambda x: classifier_pays(x))

# df

In [15]:
# def calculate_distributor_scores(df):
#     # 1. Fréquence de distribution
#     frequency = df['distributor'].value_counts()

#     # 2. Moyenne des spectateurs totaux
#     avg_total_spectators = df.groupby('distributor')['total_spectator'].mean()

#     # 3. Moyenne de la première semaine
#     avg_first_week = df.groupby('distributor')['first_week'].mean()

#     # 4. Moyenne inversée du classement hebdomadaire
#     avg_inv_hebdo_rank = 1 / df.groupby('distributor')['hebdo_rank'].mean()

#     # 5. Moyenne des évaluations de la presse
#     avg_rating_press = df.groupby('distributor')['rating_press'].mean()

#     # 5. Moyenne des récompenses
#     avg_award = df.groupby('distributor')['award'].mean()

#     # 6. Rendement (total_spectator / budget)
#     performance = (df.groupby('distributor')['total_spectator'].sum() / df.groupby('distributor')['budget'].sum())

#     # Compilation des scores dans un DataFrame
#     distributor_scores = pd.DataFrame({
#         'frequency': frequency,
#         'avg_total_spectators': avg_total_spectators,
#         'avg_first_week': avg_first_week,
#         'avg_inv_hebdo_rank': avg_inv_hebdo_rank,
#         'avg_rating_press': avg_rating_press,
#         'avg_award': avg_award,
#         'performance': performance
#     })
#     # Normalisation des scores
#     max_values = distributor_scores.max()
#     normalized_scores = distributor_scores / max_values

#     # Combinaison des scores normalisés
#     normalized_scores['distributor_combined_score'] = normalized_scores.mean(axis=1)

#     return normalized_scores

# # Appliquer la fonction pour calculer les scores
# distributor_scores = calculate_distributor_scores(df)

# # Joindre les scores combinés avec le DataFrame original
# df = df.join(distributor_scores['distributor_combined_score'], on='distributor')
# df

In [16]:
# def calculate_year_scores(df):
#     # 1. Fréquence de distribution
#     frequency = df['year'].value_counts()

#     # 2. Moyenne des spectateurs totaux
#     avg_total_spectators = df.groupby('year')['total_spectator'].mean()

#     # 3. Moyenne de la première semaine
#     avg_first_week = df.groupby('year')['first_week'].mean()

#     # 4. Moyenne du nombre de copies
#     avg_copies = df.groupby('year')['copies'].mean()

#     # Compilation des scores dans un DataFrame
#     distributor_scores = pd.DataFrame({
#         'frequency': frequency,
#         'avg_total_spectators': avg_total_spectators,
#         'avg_first_week': avg_first_week,
#         'avg_copies': avg_copies
#     })

#     # Normalisation des scores
#     max_values = distributor_scores.max()
#     normalized_scores = distributor_scores / max_values

#     # Combinaison des scores normalisés
#     normalized_scores['year_combined_score'] = normalized_scores.mean(axis=1)

#     return normalized_scores

# # Appliquer la fonction pour calculer les scores
# year_scores = calculate_year_scores(df)

# # Joindre les scores combinés avec le DataFrame original
# df = df.join(year_scores['year_combined_score'], on='year')
# df

In [17]:
# def calculate_country_scores(df):
#     # 1. Fréquence de distribution
#     frequency = df['country'].value_counts()

#     # 2. Moyenne des spectateurs totaux
#     avg_total_spectators = df.groupby('country')['total_spectator'].mean()

#     # 3. Moyenne de la première semaine
#     avg_first_week = df.groupby('country')['first_week'].mean()

#     # 4. Moyenne du nombre de copies
#     avg_copies = df.groupby('country')['copies'].mean()

#     # Compilation des scores dans un DataFrame
#     distributor_scores = pd.DataFrame({
#         'frequency': frequency,
#         'avg_total_spectators': avg_total_spectators,
#         'avg_first_week': avg_first_week,
#         'avg_copies': avg_copies
#     })

#     # Normalisation des scores
#     max_values = distributor_scores.max()
#     normalized_scores = distributor_scores / max_values

#     # Combinaison des scores normalisés
#     normalized_scores['country_combined_score'] = normalized_scores.mean(axis=1)

#     return normalized_scores

# # Appliquer la fonction pour calculer les scores
# country_scores = calculate_country_scores(df)

# # Joindre les scores combinés avec le DataFrame original
# df = df.join(country_scores['country_combined_score'], on='country')


In [18]:
# def classifier_distributeurs(distributeur):
#     if distributeur in distributeurs_tres_connus:
#         return 1
#     elif distributeur in distributeurs_moins_connus:
#         return 2
#     else:
#         return 3

# df['distributor'] = df['distributor'].apply(classifier_distributeurs)

# df

In [19]:
# df["classification_acteurs"].nunique()

In [20]:
df.isnull().sum()

raw_title             0
year                  0
month                 0
day                   0
director              0
date                  0
casting               0
distributor           0
genre                 0
country               0
duration              0
first_day             0
first_week            0
first_weekend         0
hebdo_rank            0
total_spectator       0
copies                0
rating_press          0
budget                0
lang                  0
award                 0
actor_list            0
total_actor_scores    0
dtype: int64

In [21]:
colonnes_numeriques = ["first_day", 'entree_annee',"rating_press","first_weekend",'budget']

df[colonnes_numeriques] = df[colonnes_numeriques].replace(-1, np.nan)

imputer = KNNImputer(n_neighbors=7)

df_imputed = imputer.fit_transform(df[colonnes_numeriques])

df[colonnes_numeriques] = df_imputed

df.head()

KeyError: "['entree_annee'] not in index"

In [ ]:
df.isnull().sum()

raw_title                     0
year                          0
month                         0
day                           0
director                      0
date                          0
casting                       0
distributor                   0
genre                         0
country                       0
duration                      0
first_day                     0
first_week                    0
first_weekend                 0
hebdo_rank                    0
total_spectator               0
copies                        0
rating_press                  0
budget                        0
lang                          0
award                         0
director_combined_score       0
entree_annee                  0
actor_list                    0
total_actor_scores            0
classification_country        0
distributor_combined_score    0
year_combined_score           0
country_combined_score        0
dtype: int64

In [ ]:
# df.nunique()

In [ ]:
# df['date'] = pd.to_datetime(df['date'])

# df['day'] = df['date'].dt.day
# df['month'] = df['date'].dt.month
# df['year'] = df['date'].dt.year

# # df.drop("date", axis=1, inplace=True)

In [ ]:
# def classify_season(month):
#     if month == 12 or month == 1 or month == 2:
#         return 'winter'
#     elif month == 3 or month == 4 or month == 5:
#         return 'spring'
#     elif month == 6 or month == 7 or month == 8:
#         return 'summer'
#     elif month == 9 or month == 10 or month == 11:
#         return 'autumn'


# df['season'] = df['month'].apply(classify_season)


In [ ]:
# def classify_month_name(month):
#     if month == 12:
#         return 'december'
#     elif month == 11:
#         return 'novembre'
#     elif month == 10:
#         return 'october'
#     elif month == 9:
#         return 'september'
#     elif month == 8:
#         return 'august'
#     elif month == 7:
#         return 'july'
#     elif month == 6:
#         return 'june'
#     elif month == 5:
#         return 'may'
#     elif month == 4:
#         return 'april'
#     elif month == 3:
#         return 'march'
#     elif month == 2:
#         return 'february'
#     elif month == 1:
#         return 'january'



# df['month_name'] = df['month'].apply(classify_month_name)

In [ ]:
# holiday_dates = SchoolHolidayDates()

# df['is_holiday'] = df.apply(lambda row: holiday_dates.is_holiday_for_zone(datetime.date(row['year'], row['month'], row['day']), 'B'), axis=1)
# df['is_holiday'] = df['is_holiday'].astype(int)
# df

,raw_title,year,month,day,director,date,casting,distributor,genre,country,duration,first_day,first_week,first_weekend,hebdo_rank,total_spectator,copies,rating_press,budget,lang,award,director_combined_score,entree_annee,actor_list,total_actor_scores,classification_country,distributor_combined_score,year_combined_score,country_combined_score,season,month_name,is_holiday
0,Bienvenue chez les Ch'tis,2008,2,27,"""dany boon""",2008-02-27,"[""kad merad"", ""dany boon"", ""zoe felix""]","[""pathe renn productions"", ""hirsch"", ""les prod...",comedie,france,6360,558359.000000,4378720,3.586497e+06,1,20489303,793,7.100000,1.100000e+07,"[""francais""]",5,0.467100,190.3,"[""kad merad"", ""dany boon"", ""zoe felix""]",0.559847,2,0.553998,0.863258,0.723704,winter,february,0
1,Les Bronzés 3: Amis pour la vie,2006,2,1,"""patrice leconte""",2006-02-01,"[""josiane balasko"", ""michel blanc"", ""marieanne...","[""les films christian fechner"", ""tf1 films pro...",comedie,france,5820,537882.000000,3906694,3.235559e+06,1,10355930,950,6.557143,3.500000e+07,"[""anglais"", ""italien"", ""francais""]",1,0.243394,188.8,"[""josiane balasko"", ""michel blanc"", ""mariean...",0.603943,2,0.341859,0.885471,0.723704,winter,february,0
2,Taxi 2,2000,3,29,"""gerard krawczyk""",2000-03-29,"[""samy naceri"", ""frederic diefenthal"", ""emma w...","[""arp selection"", ""canal"", ""leeloo productions""]",action,france,5400,801922.000000,3478850,2.951255e+06,1,10345901,831,6.500000,7.000000e+07,"[""francais"", ""japonais"", ""allemand""]",1,0.384457,165.8,"[""samy naceri"", ""frederic diefenthal"", ""emma...",0.434469,2,0.443038,0.796253,0.723704,spring,march,0
3,La Revanche des Sith,2005,5,18,"""george lucas""",2005-05-18,"[""hayden christensen"", ""natalie portman"", ""ewa...","[""lucasfilm"", ""mestiere cinema"", ""pandora films""]",action,etatsunis,8760,641799.000000,3303005,2.878764e+06,1,7247809,938,7.600000,1.130000e+08,"[""anglais""]",29,0.507118,175.6,"[""hayden christensen"", ""natalie portman"", ""e...",0.983180,3,0.444225,0.834583,0.923321,spring,may,0
4,Le Roi Lion (2019),2019,7,17,"""jon favreau""",2019-07-17,"[""donald glover"", ""beyonce"", ""seth rogen""]","[""walt disney pictures"", ""fairview entertainme...",animation,etatsunis,7080,630478.000000,3252896,2.559370e+06,1,10017995,680,6.800000,2.600000e+08,"[""anglais"", ""espagnol""]",21,0.305411,213.2,"[""donald glover"", ""beyonce"", ""seth rogen""]",0.524373,3,0.446294,0.850539,0.923321,summer,july,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2653,La Prisonnière espagnole,1998,1,7,"""david mamet""",1998-01-07,"[""steve martin"", ""ben gazzara"", ""campbell scott""]","[""jasmine productions inc"", ""jean doumanian pr...",drame,etatsunis,6600,28275.714286,49321,3.109739e+05,13,196895,49,7.200000,1.000000e+07,"[""anglais""]",3,0.138547,170.6,"[""steve martin"", ""ben gazzara"", ""campbell sc...",0.292473,3,0.140102,0.691216,0.923321,winter,january,0
2660,Drôles de petites bêtes,2017,12,13,"""arnaud bouron""",2017-12-13,"[""virginie efira"", ""kev adams"", ""emmanuel curt...","[""bidibul productions"", ""france 3"", ""on animat...",animation,france,5280,12587.000000,48816,4.449200e+04,16,259457,267,4.600000,3.456571e+07,"[""francais""]",1,0.037450,209.4,"[""virginie efira"", ""kev adams"", ""emmanuel cu...",0.895385,2,-0.153546,0.850278,0.723704,winter,december,0
2663,Shortbus,2006,11,8,"""john cameron mitchell""",2006-11-08,"[""sookyin lee"", ""peter stickles"", ""pj deboy""]","[""thinkfilm"", ""fortissimo films"", ""q television""]",comedie,etatsunis,6120,6226.000000,48535,3.814600e+04,16,123644,86,6.400000,2.000000e+06,"[""anglais""]",7,0.126517,188.8,"[""sookyin lee"", ""peter stickles"", ""pj deboy""]",0.246057,3,0.126877,0.885471,0.923321,autumn,novembre,0
2669,La Grande Bellezza,2013,5,22,"""paolo sorrentino""",2013-05-22,"[""toni servillo"", ""carlo verdone"", ""sabrina fe...","[""indigo film"", ""medusa 

In [ ]:
df.to_csv('Dataset_analyse.csv', index=False)
df

,raw_title,year,month,day,director,date,casting,distributor,genre,country,duration,first_day,first_week,first_weekend,hebdo_rank,total_spectator,copies,rating_press,budget,lang,award,director_combined_score,entree_annee,actor_list,total_actor_scores,classification_country,distributor_combined_score,year_combined_score,country_combined_score,season,month_name,is_holiday
0,Bienvenue chez les Ch'tis,2008,2,27,"""dany boon""",2008-02-27,"[""kad merad"", ""dany boon"", ""zoe felix""]","[""pathe renn productions"", ""hirsch"", ""les prod...",comedie,france,6360,558359.000000,4378720,3.586497e+06,1,20489303,793,7.100000,1.100000e+07,"[""francais""]",5,0.467100,190.3,"[""kad merad"", ""dany boon"", ""zoe felix""]",0.559847,2,0.553998,0.863258,0.723704,winter,february,0
1,Les Bronzés 3: Amis pour la vie,2006,2,1,"""patrice leconte""",2006-02-01,"[""josiane balasko"", ""michel blanc"", ""marieanne...","[""les films christian fechner"", ""tf1 films pro...",comedie,france,5820,537882.000000,3906694,3.235559e+06,1,10355930,950,6.557143,3.500000e+07,"[""anglais"", ""italien"", ""francais""]",1,0.243394,188.8,"[""josiane balasko"", ""michel blanc"", ""mariean...",0.603943,2,0.341859,0.885471,0.723704,winter,february,0
2,Taxi 2,2000,3,29,"""gerard krawczyk""",2000-03-29,"[""samy naceri"", ""frederic diefenthal"", ""emma w...","[""arp selection"", ""canal"", ""leeloo productions""]",action,france,5400,801922.000000,3478850,2.951255e+06,1,10345901,831,6.500000,7.000000e+07,"[""francais"", ""japonais"", ""allemand""]",1,0.384457,165.8,"[""samy naceri"", ""frederic diefenthal"", ""emma...",0.434469,2,0.443038,0.796253,0.723704,spring,march,0
3,La Revanche des Sith,2005,5,18,"""george lucas""",2005-05-18,"[""hayden christensen"", ""natalie portman"", ""ewa...","[""lucasfilm"", ""mestiere cinema"", ""pandora films""]",action,etatsunis,8760,641799.000000,3303005,2.878764e+06,1,7247809,938,7.600000,1.130000e+08,"[""anglais""]",29,0.507118,175.6,"[""hayden christensen"", ""natalie portman"", ""e...",0.983180,3,0.444225,0.834583,0.923321,spring,may,0
4,Le Roi Lion (2019),2019,7,17,"""jon favreau""",2019-07-17,"[""donald glover"", ""beyonce"", ""seth rogen""]","[""walt disney pictures"", ""fairview entertainme...",animation,etatsunis,7080,630478.000000,3252896,2.559370e+06,1,10017995,680,6.800000,2.600000e+08,"[""anglais"", ""espagnol""]",21,0.305411,213.2,"[""donald glover"", ""beyonce"", ""seth rogen""]",0.524373,3,0.446294,0.850539,0.923321,summer,july,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2653,La Prisonnière espagnole,1998,1,7,"""david mamet""",1998-01-07,"[""steve martin"", ""ben gazzara"", ""campbell scott""]","[""jasmine productions inc"", ""jean doumanian pr...",drame,etatsunis,6600,28275.714286,49321,3.109739e+05,13,196895,49,7.200000,1.000000e+07,"[""anglais""]",3,0.138547,170.6,"[""steve martin"", ""ben gazzara"", ""campbell sc...",0.292473,3,0.140102,0.691216,0.923321,winter,january,0
2660,Drôles de petites bêtes,2017,12,13,"""arnaud bouron""",2017-12-13,"[""virginie efira"", ""kev adams"", ""emmanuel curt...","[""bidibul productions"", ""france 3"", ""on animat...",animation,france,5280,12587.000000,48816,4.449200e+04,16,259457,267,4.600000,3.456571e+07,"[""francais""]",1,0.037450,209.4,"[""virginie efira"", ""kev adams"", ""emmanuel cu...",0.895385,2,-0.153546,0.850278,0.723704,winter,december,0
2663,Shortbus,2006,11,8,"""john cameron mitchell""",2006-11-08,"[""sookyin lee"", ""peter stickles"", ""pj deboy""]","[""thinkfilm"", ""fortissimo films"", ""q television""]",comedie,etatsunis,6120,6226.000000,48535,3.814600e+04,16,123644,86,6.400000,2.000000e+06,"[""anglais""]",7,0.126517,188.8,"[""sookyin lee"", ""peter stickles"", ""pj deboy""]",0.246057,3,0.126877,0.885471,0.923321,autumn,novembre,0
2669,La Grande Bellezza,2013,5,22,"""paolo sorrentino""",2013-05-22,"[""toni servillo"", ""carlo verdone"", ""sabrina fe...","[""indigo film"", ""medusa 

In [ ]:
df.isnull().sum()

raw_title                     0
year                          0
month                         0
day                           0
director                      0
date                          0
casting                       0
distributor                   0
genre                         0
country                       0
duration                      0
first_day                     0
first_week                    0
first_weekend                 0
hebdo_rank                    0
total_spectator               0
copies                        0
rating_press                  0
budget                        0
lang                          0
award                         0
director_combined_score       0
entree_annee                  0
actor_list                    0
total_actor_scores            0
classification_country        0
distributor_combined_score    0
year_combined_score           0
country_combined_score        0
season                        0
month_name                    0
is_holid